In [39]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [40]:
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
ckey = 'o7qidRODRWm5u07PtAWrCpSqf'
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV'
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc'
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret, 'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])

api = tweepy.API(auth)

In [41]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            api = tweepy.API(auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
            DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
            DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(api.search_tweets, q=aranan, result_type="new", geocode="55.0000,4.0000,1000km",lang='tr')
            results=[]
            for item in cursor.items(100):
                    results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            getTweets.TweetTemizle(DataSet)
            DataSet.to_csv('tw.csv',index=False)
        except Exception as err:
            print('Error: {}'.format(err))
                  
    def Ceviri(DataSet):
        try:
            print ("Tweetler Çeviriliyor...")
            for yazi in range(len(DataSet["Tweet"])):
                Ceviri_Blob=TextBlob(DataSet["Tweet"][yazi])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                DataSet['Tweet_Ceviri'][yazi]=Eng_Blob
            print ("Tweetler Çevirildi!")
            return DataSet        
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TweetTemizle(Dataset):
        try:
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
            for i in range(len(Dataset["Tweet_Degistirilmis"])):        
                Dataset["Tweet_Degistirilmis"][i]=pattern.sub(' ', Dataset["Tweet_Degistirilmis"][i])
            print ("Tweetler Temizlendi!")
            getTweets.Ceviri(Dataset)
            return Dataset
        except Exception as err:
            print('Error: {}'.format(err))
    
    def KelimeAyristirma(DataSet):
        try:
            data_test = DataSet.copy()
            stop = stopwords.words('english')
            data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].str.lower().str.split()
            data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].apply(lambda x: [item for item in x if item not in stop])
        except Exception as err:
            print('Error: {}'.format(err))
            
    def getSubjectivity(self,text):
        self.setAuth()
        return TextBlob(text).sentiment.subjectivity
    
    def getPolarity(self,text):
        self.setAuth()
        return TextBlob(text).sentiment.polarity 


In [ ]:
def Ceviri(DataSet):
    print ("Tweetler Çeviriliyor...")
    
    for yazi in range(len(DataSet["Tweet"])):
        Ceviri_Blob=TextBlob(DataSet["Tweet"][yazi])
        Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
        DataSet['Tweet_Ceviri'][yazi]=Eng_Blob
        
    print ("Tweetler Çevirildi!")
    return DataSet

In [ ]:
def TweetTemizle(Dataset):
    print ("Tweetler Temizleniyor...")

    pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
    for i in range(len(Dataset["Tweet_Degistirilmis"])):        
        Dataset["Tweet_Degistirilmis"][i]=pattern.sub(' ', Dataset["Tweet_Degistirilmis"][i])
        
    print ("Tweetler Temizlendi!")
    Ceviri(Dataset)
    return Dataset

In [ ]:
def KelimeAyristirma(DataSet):
    data_test = DataSet.copy()
    stop = stopwords.words('english')
    data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].str.lower().str.split()
    data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].apply(lambda x: [item for item in x if item not in stop])

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity 

In [ ]:
def toDataFrame(tweets):
    print ("Veriler Kayıt Ediliyor...")
    # COnvert to data frame    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    DataSet = pd.DataFrame()


    DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
    DataSet['Tweet'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]
    DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
    DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
    DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
    DataSet['Dil'] = [tweet.lang for tweet in tweets]
    
    """"
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets] 
    users_retweeted = []    
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    
    DataSet['TweetPlace'] = [i for i in tweets_place]
    DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]
    """
    print ("Veriler Kayıt Edildi!")
    return DataSet

In [ ]:
if (not api):
    print ("Bağlantı Yapılamadı!")
    sys.exit(-1)
else:
    print ("Bağlantı Yapıldı!")
    
    aranan=input("Aranacak Kelimeyi Girin: ")
    
    print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
    
    cursor = tweepy.Cursor(api.search_tweets, q=aranan, result_type="new", geocode="55.0000,4.0000,1000km",lang='tr')
    
    results=[]
    for item in cursor.items(100): # Remove the limit to 1000
            results.append(item)
            
    print ("Kelimeye Uygun Tweetler Sorgulandı!")
    
    DataSet = toDataFrame(results)  
    TweetTemizle(DataSet)
    
    DataSet.to_csv('tw.csv',index=False)

In [42]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Tweetler Çeviriliyor...
Tweetler Çevirildi!


In [43]:
Veri=pd.read_csv('tw.csv')
Veri.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
0,1601651063245115393,@Abdquil Müslüman Türkiye diye bir şey yok. Türkiye resmi dini olmayan seküler bir cumhuriyettir.,Müslüman Türkiye diye bir şey yok Türkiye resmi dini olmayan seküler bir cumhuriyettir,@Abdquil Muslim Turkey does not have such a thing. Turkey is a non -official religious republic.,2022-12-10 18:52:25+00:00,1065349760897490945,bilge,tr
1,1601650486649634816,@Konyabuyuksehir Türkiye’nin en zalim şehirlerinden olduğunu tüm türkiye biliyor ve hâlâ devam ediyorsunuz Allah ıs… https://t.co/qqUYeq9cDZ,Türkiye nin en zalim şehirlerinden olduğunu tüm türkiye biliyor ve hâlâ devam ediyorsunuz Allah ıs,@Konyabuyuksehir is one of the most cruel cities of Turkey and you still know and still continue Allah Is ... https://t.co/qquyeq9cdz,2022-12-10 18:50:07+00:00,950489492477173760,kübra the dissident,tr
2,1601650452755410944,@Konyabuyuksehir Türkiye’nin en zalim şehirlerinden olduğunu tüm türkiye biliyor ve hâlâ devam ediyorsunuz Allah ıs… https://t.co/fYcQP9XBOa,Türkiye nin en zalim şehirlerinden olduğunu tüm türkiye biliyor ve hâlâ devam ediyorsunuz Allah ıs,"@Konyabuyuksehir is one of the most cruel cities of Turkey, and you still know that Allah is still going on.",2022-12-10 18:49:59+00:00,950489492477173760,kübra the dissident,tr
3,1601650292797231105,Türkiye'nin uluslararası arenada itibarının daha fazla zedelenmemesi için yeniden uluslararası hukuk çizgisine dönü… https://t.co/V8ISNZfQAe,Türkiye nin uluslararası arenada itibarının daha fazla zedelenmemesi için yeniden uluslararası hukuk çizgisine dönü,"In order to prevent further injuries of Turkey's international arena, it has been returned to the international line of law… https://t.co/v8ısnzfqae",2022-12-10 18:49:21+00:00,1152543579530833920,SonEr,tr
4,1601650278994169856,@yasindedes İlk Müslüman ülke Türkiye dünya 3. Sene 2002 belki bizi geçerler de o zaman final oynayan ilk Müslüman ülke deriz,İlk Müslüman ülke Türkiye dünya Sene belki bizi geçerler de o zaman final oynayan ilk Müslüman ülke deriz,"@yasindedes First Muslim Country Turkey World 3rd Year 2002, maybe they pass us, then we say the first Muslim country to play the final",2022-12-10 18:49:18+00:00,825325914,Bjk1903 🇫🇷Paris /🇹🇷 Istanbul,tr


In [44]:
Veri.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
95,1601604351616643073,@Denizde1Adam @hbarkey Türkiye kendisi İŞ’IT zaten 😅😅,Türkiye kendisi İŞ IT zaten 😅😅,@Sea1adam @hbarkey Turkey itself is already 😅😅,2022-12-10 15:46:48+00:00,1425208552528941059,Zerdüşt Kurdi,tr
96,1601603741891047425,"@necatibulak Türkiye de siyaset, Atatürk,milliyetçilik ve din üzerinden şekillendiriliyor tanımından ziyade bu üçü… https://t.co/A3LAhJFi5t",Türkiye de siyaset Atatürk milliyetçilik ve din üzerinden şekillendiriliyor tanımından ziyade bu üçü,"@necatibulak Turkey is shaped through politics, Atatürk, nationalism and religion rather than the definition of these three… https://t.co/a3lahjfi5t",2022-12-10 15:44:23+00:00,1550557335801974784,Şeref Özcan,tr
97,1601603461665423360,Cam filmi direnişi. Türkiye'de sivil toplumun gücünün siyasette eriştiği zirve. https://t.co/6N1nXcTOXh,Cam filmi direnişi Türkiye de sivil toplumun gücünün siyasette eriştiği zirve,Window film resistance. The summit in which the power of civil society in Turkey has reached politics. https://t.co/6N1Nxctoxh,2022-12-10 15:43:16+00:00,792799036860428288,Ludwig,tr
98,1601603186850422784,@RTErdogan Türkiye'yi sokmak ile Türkiye'ye sokmak aradaki aradaki fark sadece bir harf. Insan yanlış yazabilir yani. 😄,Türkiye yi sokmak ile Türkiye ye sokmak aradaki aradaki fark sadece bir harf Insan yanlış yazabilir yani 😄,@Rtterdogan Turkey to put in Turkey with the difference between the difference is just a letter. So one can write wrong. 😄,2022-12-10 15:42:10+00:00,1373733190687453188,Dorjan Yskollari,tr
99,1601601958103556097,"Erdoğan,Siyaseti bırakacağı sinyali verdi\n\n Erdoğan'ın, ""2023'te milletimizden kendi adımıza son defa istediğimiz d… https://t.co/uh3fGES4Ct",Erdoğan Siyaseti bırakacağı sinyali verdi Erdoğan ın te milletimizden kendi adımıza son defa istediğimiz d,"Erdogan gave a signal to leave politics\n\n Erdogan, ""2023 in our nation for the last time we wanted d… https://t.co/uh3fges4CT",2022-12-10 15:37:17+00:00,3306469761,Yavuz Özcan,tr


In [45]:
savedTweets = Veri
savedTweets['Subjectivity'] = savedTweets['Tweet_Ceviri'].apply(getSubjectivity)
savedTweets['Polarity'] = savedTweets['Tweet_Ceviri'].apply(getPolarity)
savedTweets.to_csv('duygu.csv')
savedTweets.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
0,1601651063245115393,@Abdquil Müslüman Türkiye diye bir şey yok. Türkiye resmi dini olmayan seküler bir cumhuriyettir.,Müslüman Türkiye diye bir şey yok Türkiye resmi dini olmayan seküler bir cumhuriyettir,@Abdquil Muslim Turkey does not have such a thing. Turkey is a non -official religious republic.,2022-12-10 18:52:25+00:00,1065349760897490945,bilge,tr,0.375000,0.000
1,1601650486649634816,@Konyabuyuksehir Türkiye’nin en zalim şehirlerinden olduğunu tüm türkiye biliyor ve hâlâ devam ediyorsunuz Allah ıs… https://t.co/qqUYeq9cDZ,Türkiye nin en zalim şehirlerinden olduğunu tüm türkiye biliyor ve hâlâ devam ediyorsunuz Allah ıs,@Konyabuyuksehir is one of the most cruel cities of Turkey and you still know and still continue Allah Is ... https://t.co/qquyeq9cdz,2022-12-10 18:50:07+00:00,950489492477173760,kübra the dissident,tr,0.750000,-0.250
2,1601650452755410944,@Konyabuyuksehir Türkiye’nin en zalim şehirlerinden olduğunu tüm türkiye biliyor ve hâlâ devam ediyorsunuz Allah ıs… https://t.co/fYcQP9XBOa,Türkiye nin en zalim şehirlerinden olduğunu tüm türkiye biliyor ve hâlâ devam ediyorsunuz Allah ıs,"@Konyabuyuksehir is one of the most cruel cities of Turkey, and you still know that Allah is still going on.",2022-12-10 18:49:59+00:00,950489492477173760,kübra the dissident,tr,0.750000,-0.250
3,1601650292797231105,Türkiye'nin uluslararası arenada itibarının daha fazla zedelenmemesi için yeniden uluslararası hukuk çizgisine dönü… https://t.co/V8ISNZfQAe,Türkiye nin uluslararası arenada itibarının daha fazla zedelenmemesi için yeniden uluslararası hukuk çizgisine dönü,"In order to prevent further injuries of Turkey's international arena, it has been returned to the international line of law… https://t.co/v8ısnzfqae",2022-12-10 18:49:21+00:00,1152543579530833920,SonEr,tr,0.166667,0.000
4,1601650278994169856,@yasindedes İlk Müslüman ülke Türkiye dünya 3. Sene 2002 belki bizi geçerler de o zaman final oynayan ilk Müslüman ülke deriz,İlk Müslüman ülke Türkiye dünya Sene belki bizi geçerler de o zaman final oynayan ilk Müslüman ülke deriz,"@yasindedes First Muslim Country Turkey World 3rd Year 2002, maybe they pass us, then we say the first Muslim country to play the final",2022-12-10 18:49:18+00:00,825325914,Bjk1903 🇫🇷Paris /🇹🇷 Istanbul,tr,0.416667,0.125


In [46]:
savedTweets.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
95,1601604351616643073,@Denizde1Adam @hbarkey Türkiye kendisi İŞ’IT zaten 😅😅,Türkiye kendisi İŞ IT zaten 😅😅,@Sea1adam @hbarkey Turkey itself is already 😅😅,2022-12-10 15:46:48+00:00,1425208552528941059,Zerdüşt Kurdi,tr,0.000000,0.0
96,1601603741891047425,"@necatibulak Türkiye de siyaset, Atatürk,milliyetçilik ve din üzerinden şekillendiriliyor tanımından ziyade bu üçü… https://t.co/A3LAhJFi5t",Türkiye de siyaset Atatürk milliyetçilik ve din üzerinden şekillendiriliyor tanımından ziyade bu üçü,"@necatibulak Turkey is shaped through politics, Atatürk, nationalism and religion rather than the definition of these three… https://t.co/a3lahjfi5t",2022-12-10 15:44:23+00:00,1550557335801974784,Şeref Özcan,tr,0.000000,0.0
97,1601603461665423360,Cam filmi direnişi. Türkiye'de sivil toplumun gücünün siyasette eriştiği zirve. https://t.co/6N1nXcTOXh,Cam filmi direnişi Türkiye de sivil toplumun gücünün siyasette eriştiği zirve,Window film resistance. The summit in which the power of civil society in Turkey has reached politics. https://t.co/6N1Nxctoxh,2022-12-10 15:43:16+00:00,792799036860428288,Ludwig,tr,0.000000,0.0
98,1601603186850422784,@RTErdogan Türkiye'yi sokmak ile Türkiye'ye sokmak aradaki aradaki fark sadece bir harf. Insan yanlış yazabilir yani. 😄,Türkiye yi sokmak ile Türkiye ye sokmak aradaki aradaki fark sadece bir harf Insan yanlış yazabilir yani 😄,@Rtterdogan Turkey to put in Turkey with the difference between the difference is just a letter. So one can write wrong. 😄,2022-12-10 15:42:10+00:00,1373733190687453188,Dorjan Yskollari,tr,0.900000,-0.5
99,1601601958103556097,"Erdoğan,Siyaseti bırakacağı sinyali verdi\n\n Erdoğan'ın, ""2023'te milletimizden kendi adımıza son defa istediğimiz d… https://t.co/uh3fGES4Ct",Erdoğan Siyaseti bırakacağı sinyali verdi Erdoğan ın te milletimizden kendi adımıza son defa istediğimiz d,"Erdogan gave a signal to leave politics\n\n Erdogan, ""2023 in our nation for the last time we wanted d… https://t.co/uh3fges4CT",2022-12-10 15:37:17+00:00,3306469761,Yavuz Özcan,tr,0.066667,0.0
